In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [14]:
df=pd.read_json('reddit_jokes.json')

In [17]:
df.head()

,body,id,score,title
0,"Now I have to say ""Leroy can you please paint ...",5tz52q,1,I hate how you cant even say black paint anymore
1,Pizza doesn't scream when you put it in the ov...,5tz4dd,0,What's the difference between a Jew in Nazi Ge...
2,...and being there really helped me learn abou...,5tz319,0,I recently went to America....
3,A Sunday school teacher is concerned that his ...,5tz2wj,1,"Brian raises his hand and says, “He’s in Heaven.”"
4,He got caught trying to sell the two books to ...,5tz1pc,0,You hear about the University book store worke...


In [18]:
text=df['body']

In [21]:
import re
def normalize_text(text):

    text=text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(pic\.twitter\.com/[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    text = re.sub('#([^\s]+)', '', text)
    text = re.sub('[:;>?<=*+()&,\-#!$%\{˜|\}\[^_\\@\]1234567890’‘]',' ', text)
    text = re.sub('[\d]','', text)
    text = text.replace(".", '')
    text = text.replace("'", '')
    text = text.replace("`", '')
    text = text.replace("'s", '')
    text = text.replace("/", ' ')
    text = text.replace("\"", ' ')
    text = text.replace("\\", '')
    re.sub(' +', ' ', text)
    text=text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    #normalize some utf8 encoding
    text = text.replace("\x9d",'').replace("\x8c",'')
    text = text.replace("\xa0",'')
    text = text.replace("\x9d\x92", '').replace("\x9a\xaa\xf0\x9f\x94\xb5", '').replace("\xf0\x9f\x91\x8d\x87\xba\xf0\x9f\x87\xb8", '').replace("\x9f",'').replace("\x91\x8d",'')
    text = text.replace("\xf0\x9f\x87\xba\xf0\x9f\x87\xb8",'').replace("\xf0",'').replace('\xf0x9f','').replace("\x9f\x91\x8d",'').replace("\x87\xba\x87\xb8",'')
    text = text.replace("\xe2\x80\x94",'').replace("\x9d\xa4",'').replace("\x96\x91",'').replace("\xe1\x91\xac\xc9\x8c\xce\x90\xc8\xbb\xef\xbb\x89\xd4\xbc\xef\xbb\x89\xc5\xa0\xc5\xa0\xc2\xb8",'')
    text = text.replace("\xe2\x80\x99s", "").replace("\xe2\x80\x98", '').replace("\xe2\x80\x99", '').replace("\xe2\x80\x9c", "").replace("\xe2\x80\x9d", "")
    text = text.replace("\xe2\x82\xac", "").replace("\xc2\xa3", "").replace("\xc2\xa0", "").replace("\xc2\xab", "").replace("\xf0\x9f\x94\xb4", "").replace("\xf0\x9f\x87\xba\xf0\x9f\x87\xb8\xf0\x9f", "")
    text =  re.sub(r"\b[a-z]\b", "", text)
    text=re.sub( '\s+', ' ', text).strip()
    
    text=re.sub(r'\.+', ".", text)
    text=re.sub(r'\.\.+', ' ', text).replace('.', '')
    # Replace multiple dots with space
    text = re.sub('\.\.+', ' ', text) 
    # Remove single dots
    text = re.sub('\.', '', text)
    text = re.sub(r'\.{2,}', ' ', text)
    text = re.sub(r'\.{1}', '', text)
    
    return text

In [33]:
normed_text=[]
for sentence in df['body']:
    normed_text.append(normalize_text(sentence))

In [56]:
#Number of unique words
import collections
words=" ".join(normed_text).split() 
count= collections.Counter(words).most_common()
print ("Word count", len(count))

Word count 81101


In [74]:
#Build a vocabulary
unique_words = [i[0] for i in count]
vocabulary={}
for i, word in enumerate(unique_words):
    vocabulary[word]=i
vocab_size=len(vocabulary)

In [76]:
# Make word indexes
word_index=[]
for word in words:
    word_index.append(vocabulary[word])

In [83]:
# Make skimgram pairs
window = 1
skipgram_words=[]
for i in range(1, len(words)-1):
    skipgram_words.append([words[i], words[i-1]])
    skipgram_words.append([words[i], words[i+1]])
    
skipgram_indexes=[]
for i in range(1, len(word_index)-1):
    skipgram_indexes.append([word_index[i], word_index[i-1]])
    skipgram_indexes.append([word_index[i], word_index[i+1]])

In [89]:
def get_batch(size):
    assert size<len(skipgram_indexes)
    X=[]
    Y=[]
    rdm = np.random.choice(range(len(skipgram_indexes)),size,replace=False)
    
    for r in rdm:
        X.append(skipgram_indexes[r][0])
        Y.append(skipgram_indexes[r][1])
    return X , Y

In [91]:
embed_size=2
x=tf.placeholder(shape=[vocab_size, 1], dtype=tf.float32, name="x")
y=tf.placeholder(shape=[vocab_size, 1], dtype=tf.float32, name="y")
W1 = tf.Variable(tf.random_uniform([vocab_size,embed_size],-1.0,1.0))
W2= tf.Variable(tf.random_uniform([embed_size,vocab_size],-1.0,1.0))
h1=tf.matmul(tf.transpose(W1), x)
h2=tf.matmul(tf.transpose(W2), h1)
Loss = -tf.matmul(tf.transpose(h2), y)+tf.log(tf.reduce_sum(tf.exp(h2), axis=1))
optimizer = tf.train.AdamOptimizer(1e-1).minimize(Loss)

In [ ]:
epochs=10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        
        batch_inputs, batch_labels = get_batch(batch_size)
        _,loss_val=sess.run([optimizer,loss],feed_dict = {X : batch_inputs, Y : batch_labels })
        
        if epoch % 1000 == 0:
            print("Loss at ", epoch, loss_val) # Report the loss
    
    # Final embeddings are ready for you to use. Need to normalize for practical use
    trained_embeddings = embeddings.eval()